In [2]:

from scipy.spatial import distance
from imutils import face_utils
import numpy as np
import time
import dlib
import cv2
from gaze_tracking import GazeTracking


#Minimum threshold of eye aspect ratio below which alarm is triggerd
EYE_ASPECT_RATIO_THRESHOLD = 0.3

#Minimum consecutive frames for which eye ratio is below threshold for alarm to be triggered
EYE_ASPECT_RATIO_CONSEC_FRAMES = 50

#COunts no. of consecutuve frames below threshold value
COUNTER = 0

#Load face cascade which will be used to draw a rectangle around detected faces.
face_cascade = cv2.CascadeClassifier("haarcascades/haarcascade_frontalface_default.xml")

#This function calculates and return eye aspect ratio
def eye_aspect_ratio(eye):
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])

    ear = (A+B) / (2*C)
    return ear

#Load face detector and predictor, uses dlib shape predictor file
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

#Extract indexes of facial landmarks for the left and right eye
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS['left_eye']
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS['right_eye']


frame = cv2.imread('C:\\Users\\Abhilasha\\Pictures\\hack36_train\\right\\aswat1_right.jpg')
frame = cv2.flip(frame,1)
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
gaze = GazeTracking()

gaze.refresh(frame)
frame = gaze.annotated_frame()
faces = detector(gray, 0)

#Detect faces through haarcascade_frontalface_default.xml
face_rectangle = face_cascade.detectMultiScale(gray, 1.3, 5)

#Draw rectangle around each face detected
for (x,y,w,h) in face_rectangle:
    cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)

    #Detect facial points
for face in faces:

    shape = predictor(gray, face)
    shape = face_utils.shape_to_np(shape)

    #Get array of coordinates of leftEye and rightEye
    leftEye = shape[lStart:lEnd]
    rightEye = shape[rStart:rEnd]
    print(leftEye)
    print(rightEye)
    #Calculate aspect ratio of both eyes
    leftEyeAspectRatio = eye_aspect_ratio(leftEye)
    rightEyeAspectRatio = eye_aspect_ratio(rightEye)

    eyeAspectRatio = (leftEyeAspectRatio + rightEyeAspectRatio) / 2

    #Use hull to remove convex contour discrepencies and draw eye shape around eyes
    leftEyeHull = cv2.convexHull(leftEye)
    rightEyeHull = cv2.convexHull(rightEye)
    a=gaze.pupil_left_coords()
    b=gaze.pupil_right_coords()
    
    cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
    cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
    cv2.circle(frame, a, 3, (0, 255, 0), -1)
    cv2.circle(frame, b, 3, (0, 255, 0), -1)
    print(a, b)
      
    
cv2.imwrite('zzz.jpg', frame)
   




[[561 273]
 [576 261]
 [594 261]
 [608 271]
 [594 276]
 [577 275]]
[[445 271]
 [459 261]
 [477 261]
 [491 273]
 [476 276]
 [458 276]]
(479, 269) (598, 268)


True